# Personal Investment Analysis

In [2]:
using JPPF, PyPlot, DataFrames, NPFinancial, Dates, KeywordSearch, Random
using StatsPlots, Formatting, Dates, PrettyTables, XLSX, PlotThemes, MarketData
ht("<H3>Personal Investment Analysis<H3>")
ht("Report Date: $(today())")

┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall /home/steve/.julia/packages/PyCall/SBNSg/src/PyCall.jl:719
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda /home/steve/.julia/packages/Conda/xMClC/src/Conda.jl:127


done
Solving environment: ...working... 

done



## Package Plan ##

  environment location: /home/steve/.julia/conda/3/x86_64

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.8             |       h166bdaf_0         578 KB  conda-forge
    attr-2.5.1                 |       h166bdaf_1          69 KB  conda-forge
    brotli-1.0.9               |       h166bdaf_8          18 KB  conda-forge
    brotli-bin-1.0.9           |       h166bdaf_8          20 KB  conda-forge
    cairo-1.16.0               |    hbbf8b49_1016         1.1 MB  conda-forge
    contourpy-1.1.0            |  py310hd41b1e2_0         216 KB  conda-forge
    cycler-0.11.0              |     pyhd8ed1ab_0          10 KB  conda-forge
    dbus-1.13.6                |       h5008d03_3         604 KB  conda-forge
    expat-2.5.0                |       hcb278e6_1         134 KB  conda-forge
    font-ttf-dejavu-sans-mono-



lerc-4.0.0           | 275 KB    |            |   0% 
lz4-c-1.9.4          | 140 KB    |            |   0% 

libpng-1.6.39        | 276 KB    |            |   0% 




...





qt-main-5.15.8       | 57.8 MB   | ###6       |  36% 





qt-main-5.15.8       | 57.8 MB   | ###6       |  36% 





qt-main-5.15.8       | 57.8 MB   | ###6       |  36% 





qt-main-5.15.8       | 57.8 MB   | ###6       |  36% 





qt-main-5.15.8       | 57.8 MB   | ###6       |  36% 

In [2]:
dataDir="../data"
invSheet=joinpath(dataDir,"Investments.xlsx")
ht("Investment input: $invSheet")
invTab="Investments"
transSheet=joinpath(dataDir, "2021-01-01 thru 2021-08-05 transactions.xlsx")
ht("TransactionInput input: $transSheet")
inv=readTab(invSheet, invTab)
stockList=select(inv, [:Code, :Type]) 
stockList=filter(:Type => t -> t == "Stock", stockList)
stockList=stockList[!, :Code]
holdings=Symbol.(setdiff(inv.Code, ["House", "FidelityHP"]))
trans=readTab(transSheet, 1);
#formatters = (v, i, j) -> (j in [6] ) ? format(round(Int, v), commas=true,parens=true) : v
#pretty_table(inv; backend = Val(:html), nosubheader=true)

Investment input: ../data/Investments.xlsx

TransactionInput input: ../data/2021-01-01 thru 2021-08-05 transactions.xlsx

,Date,Account,Description
,String,String,String
1,2021-08-02,Brokeragelink - Ending in 9921,Dividend Received As Of 07/31/2021 Jpmorgan Chase & Co
2,2021-08-02,Brokeragelink - Ending in 9921,Reinvestment As Of 07/31/2021 Jpmorgan Chase & Co
3,2021-08-02,Brokeragelink - Ending in 9921,Transferred From To Brokerage Option
4,2021-07-30,Indiana Farm 401k - Ending in 1294,Fidelity Govt Income - Dividend
5,2021-07-30,Indiana Farm 401k - Ending in 1294,Fid Govt Mmkt K6 - Dividend
6,2021-07-30,Indiana Farm 401k - Ending in 1294,Fid 500 Index - Contributions
7,2021-07-30,Indiana Farm 401k - Ending in 1294,Brokeragelink - Contributions
8,2021-07-30,Brokeragelink - Ending in 9921,Reinvestment Fidelity Government Cash Reserves
9,2021-07-30,Brokeragelink - Ending in 9921,Dividend Received Fidelity Government Cash Reserves


## Historical Analysis 

In [3]:
#Find sold investments
sold=  filter(:Amount => t -> t<(0.0), trans)
bought=filter(:Amount => t -> t>(0.0), trans)
#profit=outerjoin(sold, bought, on=:InvestmentID, makeunique=true)
pretty_table(bought; backend = Val(:html), nosubheader=true)
#profit=select(profit, [1,3,4,5,8])
#rename!(profit, ["ID", "AmountSold", "SharesSold", "DateSold", "Basis"])

Date,Account,Description,Category,Tags,Amount
2021-08-02,Brokeragelink - Ending in 9921,Dividend Received As Of 07/31/2021 Jpmorgan Chase & Co,Investment Income,missing,22.5
2021-08-02,Brokeragelink - Ending in 9921,Transferred From To Brokerage Option,Transfers,missing,956.21
2021-07-30,Indiana Farm 401k - Ending in 1294,Fidelity Govt Income - Dividend,Securities Trades,missing,7.69
2021-07-30,Indiana Farm 401k - Ending in 1294,Fid Govt Mmkt K6 - Dividend,Securities Trades,missing,0.1
2021-07-30,Indiana Farm 401k - Ending in 1294,Fid 500 Index - Contributions,Retirement Contributions,missing,956.23
2021-07-30,Indiana Farm 401k - Ending in 1294,Brokeragelink - Contributions,Retirement Contributions,missing,956.21
2021-07-30,Brokeragelink - Ending in 9921,Dividend Received Fidelity Government Cash Reserves,Investment Income,missing,0.08
2021-07-30,Brokeragelink - Ending in 9921,Dividend Received As Of 07/28/2021 Bank Of Nova Scotia Com Npv Isin #ca064,Investment Income,missing,7.16
2021-07-30,Brokeragelink - Ending in 9921,Dividend Received As Of 07/28/2021 Canadian Imperial Bank Of Commerce Com,Investment Income,missing,23.25
2021-07-19,Brokeragelink - Ending in 9921,Transferred From To Brokerage Option,Transfers,missing,956.21


In [4]:
opts=PolyOpts("SgTZRxROKKu6NrZ9SMSc60myLze5jgOa", DataFrame)
function getstock_daily(opts, holding, day)
  try 
#    stocks_daily_open_close(opts, holding, day)
     stocks_previous_close(opts,holding)
  catch 
    try 
#      stocks_daily_open_close(opts, holding, day)
    sleep(5)
    stocks_previous_close(opts,holding)
    catch
      display("Unable to get price for $holding on $day")
      missing
    end
  end
end
stocklist_daily(opts, holdings, day=today())=vcat(map(h-> getstock_daily(opts, h, day), holdings)...);


LoadError: UndefVarError: PolyOpts not defined

In [13]:
function lastBusinessDay(d)
  dow=Dates.dayofweek(d)
  if dow in [2,3,4,5,6] #T-Sa
    offset=1
  elseif dow == 1 #M
    offset=3
  else
    offset=2 #Su
  end
  d-Dates.Day(offset)
end
function getHLOC(ticker, date)
  startDate=lastBusinessDay(date)
  try 
    df=DataFrame(yahoo(ticker, YahooOpt(period1=startDate, period2=startDate)))
  catch
    display("Unable to find ticker $ticker for $(string(startDate))")
    return missing
  end
  df=hcat(df, [ticker])
  DataFrames.rename!(df, 8 => "Ticker")
end

getHLOC (generic function with 1 method)

In [16]:
date=DateTime(2021,8,20)
yahoo("MRNA", YahooOpt(period1=date))

2×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 2021-08-20 to 2021-08-20
│            │ Open     │ High   │ Low    │ Close  │ AdjClose │ Volume      │
├────────────┼──────────┼────────┼────────┼────────┼──────────┼─────────────┤
│ 2021-08-20 │ 374.548  │ 387.49 │ 369.26 │ 382.98 │ 382.98   │ 1.31108e7   │
│ 2021-08-20 │ 374.5475 │ 387.4  │ 369.26 │ 382.98 │ 382.98   │ 1.3216681e7 │

In [17]:
stocksYesterday=vcat(map(x -> getHLOC(x, today()), stockList)...)
pretty_table(stocksYesterday; backend = Val(:html), nosubheader=true)

"Unable to find ticker MRNA for 2021-08-20"

"Unable to find ticker JPM for 2021-08-20"

"Unable to find ticker LPX for 2021-08-20"

"Unable to find ticker INTC for 2021-08-20"

"Unable to find ticker CM for 2021-08-20"

...

"Unable to find ticker RY for 2021-08-20"

"Unable to find ticker FCEL for 2021-08-20"

"Unable to find ticker BNS for 2021-08-20"

"Unable to find ticker WY for 2021-08-20"

Col. 1
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing


In [2]:
using KeywordSearch, Random
d1="Dividend Received As Of 07/31/2021 Jpmorgan Chase & Co"
s2="JPMorgan chase"
fuzzy_query=FuzzyQuery(s2)
m=match(fuzzy_query, d1)
display(m)
explain(m)


LoadError: MethodError: no method matching match(::FuzzyQuery{DamerauLevenshtein{Nothing}, Int64}, ::String)
[0mClosest candidates are:
[0m  match([91m::Regex[39m, ::Union{SubString{String}, String}, [91m::Integer[39m) at regex.jl:295
[0m  match([91m::Regex[39m, ::Union{SubString{String}, String}, [91m::Integer[39m, [91m::UInt32[39m) at regex.jl:295
[0m  match([91m::Regex[39m, ::AbstractString) at regex.jl:316
[0m  ...